In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
model = AutoModel.from_pretrained("roberta-large")
tokenizer = AutoTokenizer.from_pretrained("roberta-large")

In [ ]:
import csv
def _read_tsv(cls, input_file, quotechar=None):
    """Reads a tab separated value file."""
    with open(input_file, "r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
        lines = []
        for line in reader:
            if sys.version_info[0] == 2:
                line = list(unicode(cell, 'utf-8') for cell in line)
            lines.append(line)
        return lines

lines = _read_tsv("_", "dataset/WN18RR/test.tsv")

import random
random.shuffle(lines)
len_lines = len(lines)

for i in range(0,10):
    with open(f"dataset/WN18RR/test{i+1}.tsv", "w", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter="\t")
        writer.writerows(lines[int(len_lines/10)*i : int(len_lines/10)*(i+1) if i != 9 else len_lines])


In [ ]:
lines[0]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from models import KGBERTGetLabelWord
from lit_models import TransformerLitModel


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = KGBERTGetLabelWord.from_pretrained("bert-base-uncased")
model_path = "/home/xx/AI/kgc/output/epoch=3-Eval\/acc=0.00-v1.ckpt"
hhmodel = TransformerLitModel(args={}, model=model)
hhmodel.load_state_dict(torch.load(model_path)['state_dict'])
model = hhmodel.model

In [ ]:
# select the FB15K237 test set

with open("./dataset/FB15k-237/test.tsv", "r") as file:
    lines = file.readlines()

import random

random.seed(666)
random.shuffle(lines)


with open("./dataset/FB15k-237/selected_test.tsv", "w") as file:
    file.writelines(lines[:2000])


In [ ]:
lines

In [ ]:
# convert umls dataset to the openKE format

# create the entity2id.txt

with open("./dataset/umls/entities.txt", "r") as file:
    lines = file.readlines()

with open("./dataset/umls/entity2id.txt", "w") as file:
    total = len(lines)
    cnt = 0
    file.write(f"{total}\n")
    for line in lines:
        line = line.replace("\n","")
        file.write(f"{line}\t{cnt}\n")
        cnt += 1

In [ ]:
# create the relation2id.txt

with open("./dataset/umls/relations.txt", "r") as file:
    lines = file.readlines()

with open("./dataset/umls/relation2id.txt", "w") as file:
    total = len(lines)
    cnt = 0
    file.write(f"{total}\n")
    for line in lines:
        line = line.replace("\n","")
        file.write(f"{line}\t{cnt}\n")
        cnt += 1

In [ ]:
# make constran.txt

with open("./dataset/umls/relation2id.txt", "r") as file:
    lines = file.readlines()
    num_relation = int(lines[0])
with open("./dataset/umls/relation2id.txt", "r") as file:
    line = file.readlines()[0]
    num_entity = int(line)

with open("./dataset/type_constrain.txt", "w") as file:
    file.write(f"{num_relation}\n")
    l = "\t".join([str(_) for _ in range(num_entity)])
    for i in range(num_relation):
        file.write(f"{i}\t{l}\n")
        file.write(f"{i}\t{l}\n")


In [3]:
from transformers import BartForConditionalGeneration, AutoTokenizer, GPT2LMHeadModel, BartModel, T5ForConditionalGeneration,AutoModelForSeq2SeqLM, AutoConfig
import torch
import csv
# model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
model_name_or_path = "facebook/mbart-large-cc25"
config = AutoConfig.from_pretrained("facebook/mbart-large-cc25")
config.use_prefix = False
config.force_bos_token_to_be_generated = True
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
device = "cuda:1"

AttributeError: 'MBartConfig' object has no attribute 'preseqlen'

In [ ]:
model = model.to(device)
def get_mask_word(text, model, tokenizer):
    origin_tokens = text.split()
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {k:v.to(device) for k,v in inputs.items()}
    output = model.generate(
        **inputs, 
        num_beams=10,
        num_return_sequences=1
    )
    _txt = tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
    # for t in origin_tokens:
    #     _txt = _txt.replace(t, "<e>")
    return _txt


In [ ]:
config.force_bos_token_to_be_generated = True

In [ ]:
get_mask_word("<mask> says I love Japan. But <mask>", model, tokenizer)
from data.processor import KGProcessor
processor = KGProcessor()
path = "dataset/umls"
examples = processor.get_train_examples(path)
# inputs

In [ ]:
from collections import defaultdict, Counter
relation2pattern = defaultdict(list)
from tqdm import tqdm
import re
i = 0
for example in tqdm(examples):
    i += 1
    if i%2: continue
    head,relation,tail = example.text_a,example.text_b,example.text_c
    text = " <mask> " + head + " <mask> " + relation + " <mask> " + tail + " <mask> "
    p = get_mask_word(text, model, tokenizer)
    p = re.sub(' +', '', p)
    for t_ in [head, relation, tail]:
        p = p.replace(t_, "\t")
    p = re.sub(' +', '', p)
    relation2pattern[relation].append(p)

for k in relation2pattern.keys():
    relation2pattern[k] = Counter(relation2pattern[k])

import pickle 
import os
with open(os.path.join(path, "cached_relation_pattern.pkl"), "wb") as file:
    pickle.dump(relation2pattern, file)

In [ ]:
pattern = {}
for k,v in relation2pattern.items():
    t = v.most_common(1)[0][0]
    t = [_.strip() for _ in t.split("\t")]
    t[-1] = " ."
    print(k)
    print(t)
    pattern[k] = t
    assert len(t) ==4

with open(os.path.join(path, "cached_relation_pattern.pkl"), "wb") as file:
    pickle.dump(pattern, file)

In [ ]:
len(pattern)

In [ ]:
from models.trie import Trie

label_sequence = ["ceo", "founder", "new ceo"]
# label_sequence_ids = tokenizer(label_sequence, add_special_tokens=T).input_ids
# trie = Trie([[0, 32099] + _ +[32100] for _ in label_sequence_ids])
# print(label_sequence_ids)
from models.utils import get_entity_spans_pre_processing
from models.trie import get_end_to_end_prefix_allowed_tokens_fn_hf
mention_trie = Trie([[2] + tokenizer(" type Movie").input_ids])
entity_trie = Trie([[2] + tokenizer("type Movie").input_ids])
input_sentences = ["Steve Jobs is the <mask> of Apple."]
inputs = tokenizer("Steve Jobs is the <mask> of {Apple}.", return_tensors='pt')
prefix_allowed_tokens_fn = get_end_to_end_prefix_allowed_tokens_fn_hf(
            tokenizer=tokenizer,
            sentences=get_entity_spans_pre_processing(input_sentences),
            mention_trie=mention_trie,
            candidates_trie=entity_trie
        )

In [ ]:
# 测试模型在mask上的输出能力
inputs = tokenizer(" <mask> genre  Romance Film", return_tensors="pt")
model.force_bos_token_to_be_generated = True
outputs = model.generate(**inputs,
prefix_allowed_tokens_fn=lambda batch_id, sent: entity_trie.get(sent.tolist()),
num_beams=200,
        num_return_sequences=1,
        max_length=40,
forced_bos_token_id=tokenizer.bos_token_id,
)
print(outputs)
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(outputs[0])))
outputs = model.generate(**inputs,
forced_bos_token_id=tokenizer.bos_token_id,
)
print(outputs)
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(outputs[0])))

In [ ]:
def mask_filling(text, model, tokenizer, fn, device="cpu"):
    def decode(output_ids):
        return [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in output_ids]
    inputs = tokenizer(text, return_tensors="pt").to(device)
    model = model.to(device)
    prefix_output = model.generate(
        **inputs,
        prefix_allowed_tokens_fn=fn,
        forced_bos_token_id=tokenizer.bos_token_id,
    )
    origin_output = model.generate(
        **inputs,
        num_beams=20,
        num_return_sequences=1,
        forced_bos_token_id=tokenizer.bos_token_id,
    )
    print("prefix output: \t", prefix_output)
    print("prefix output token: \y" + decode(prefix_output)[0])
    print("output: ",  origin_output)
    print("output token: " + decode(origin_output)[0])

In [ ]:
label_text = [" type Movie", " type type"]
entity_trie = Trie(
    [
    [tokenizer.eos_token_id] + _ for _ in tokenizer(label_text).input_ids
]
)
prefix_allowed_tokens_fn=lambda batch_id, sent: entity_trie.get(sent.tolist())
mask_filling("Southern California often abbreviated as SoCal . Southern California often abbreviated as <mask> .", model, tokenizer, prefix_allowed_tokens_fn)


In [ ]:
for a in entity_trie:
    print(a)

In [ ]:
path = "./dataset/SST-2/test.tsv"
examples = []
with open(path, "r") as file:
    for line in file.readlines():
        label = line.split(" ", 1)[0]
        text = line.split(" ", 1)[1]
        examples.append(dict(labels=label, text=text))

In [ ]:
input_ids = []
attention_mask = []
labels = []
for example in examples:
    inputs = tokenizer(
        example['text'],
        truncation="longest_first",
        max_length=128,
        padding="max_length",
        add_special_tokens=True,
    )
    input_ids.append(inputs.input_ids)
    attention_mask.append(inputs.attention_mask)
    labels.append(example['labels'])
from torch.utils.data import Dataset, DataLoader, TensorDataset
input_ids = torch.tensor(input_ids)
attention_mask = torch.tensor(attention_mask)
labels = torch.tensor([int(_) for _ in labels])
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=16)

In [ ]:
model_output = []
device = "cuda:1"
labels = []
model = model.to(device).eval()
for batch in dataloader:
    batch = [_.to(device) for _ in batch]
    inputs = dict(input_ids=batch[0], attention_mask=batch[1])
    outputs = model.generate(**inputs, prefix_allowed_tokens_fn=lambda batch_id, sent: trie.get(sent.tolist()),
        num_beams=200,
        num_return_sequences=1,
        max_length=10
    )
    print(outputs[0])
    for o in outputs:
        if torch.equal(torch.tensor([2, 100, 206, 24, 16, 6587, 2]).to(device), o[:10]):
            model_output += [0]
        else:
            model_output += [1]
    labels += batch[2].detach().cpu().tolist()
    

In [ ]:
cnt = 0
for a,b in zip(model_output, labels):
    if a == b: cnt += 1
print(cnt / len(model_output)) 

In [ ]:
tokenizer
model.force_bos_token_to_be_generated

In [ ]:
a[0].shape

In [ ]:

from transformers import BartTokenizer, BartForCausalLM

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForCausalLM.from_pretrained('facebook/bart-base', add_cross_attention=False)
assert model.config.is_decoder, f"{model.__class__} has to be configured as a decoder."
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

In [ ]:
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
import torch

T5_PATH = 't5-large' # "t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"

DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu') # My envirnment uses CPU

t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)
t5_config = T5Config.from_pretrained(T5_PATH)
t5_mlm = T5ForConditionalGeneration.from_pretrained(T5_PATH, config=t5_config).to(DEVICE)

# Input text

def get_words(text,):
    # text = 'Who is the founder of Huawei? It is <extra_id_0> .'

    encoded = t5_tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')
    input_ids = encoded['input_ids'].to(DEVICE)

    # Generaing 20 sequences with maximum length set to 5
    outputs = t5_mlm.generate(input_ids=input_ids, 
                            num_beams=200, num_return_sequences=20,
                            max_length=10)

    _0_index = text.index('<extra_id_0>')
    _result_prefix = text[:_0_index]
    _result_suffix = text[_0_index+12:]  # 12 is the length of <extra_id_0>

    def _filter(output, end_token='<extra_id_1>'):
        # The first token is <unk> (inidex at 0) and the second token is <extra_id_0> (indexed at 32099)
        _txt = t5_tokenizer.decode(output[2:], skip_special_tokens=False, clean_up_tokenization_spaces=False)
        if end_token in _txt:
            _end_token_index = _txt.index(end_token)
            return _txt[:] 
        else:
            return _result_prefix + _txt + _result_suffix

    results = list(map(_filter, outputs))
    return results


In [ ]:

get_words("<extra_id_0> work, prepare for crops; \"Work the soil\"; \"cultivate the land\" <extra_id_1> verb group <extra_id_2> work, make uniform; \"knead dough\"; \"work the clay until it is soft <extra_id_3>")


In [ ]:
# test the model in the SQUAD
samples = []
import json
with open("./dataset/LAMA/Squad/test.jsonl", "r") as file:
    for line in file.readlines():
        t = json.loads(line)
        samples.append(dict(text=t['masked_sentences'][0], label=t['obj_label']))

In [ ]:
# get precision@10

num_samples = len(samples)
cnt = 0
from tqdm import tqdm
for s in tqdm(samples):
    # a = get_words(s['text'].replace("[MASK]", "<extra_id_0>"))
    # result = [aa.split("<extra_id_1>")[0].strip() for aa in a[0:1]]
    txt = get_mask_word(s['text'].replace("[MASK]", "<mask>"), model, tokenizer)
    if s['label'] in txt:
        cnt += 1
print(cnt / num_samples)
# print(samples[0]['label'])
result

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


In [ ]:
model = AutoModel.from_pretrained("facebook/mbart-large-50")

In [ ]:
path = "./dataset/FB15k-237"
import pickle
with open(os.path.join(path, "cached_relation_pattern.pkl"), "rb") as file:
    a = pickle.load(file)
    


In [ ]:
for k, v in a.items():
    print(k)
    print(v.most_common(1)[0][0].split("\t"))

In [1]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  
device = "cuda:1"
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-tatoeba-en-ja")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-tatoeba-en-ja").to(device)

OSError: Can't load config for 'Helsinki-NLP/opus-tatoeba-en-ja'. Make sure that:

- 'Helsinki-NLP/opus-tatoeba-en-ja' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'Helsinki-NLP/opus-tatoeba-en-ja' is the correct path to a directory containing a config.json file



In [9]:
import os
with open("./dataset/wikidata5m/entity2text.txt", "r") as file:
    lines = file.readlines()

In [13]:
text2id = {}
cnt = 0
total = []
for line in lines:
    t = line.strip().split("\t")[0]
    assert "Q" in t
    total.append(t)
print(cnt)


0


In [15]:
total[:100]

['Q5196650',
 'Q912600',
 'Q47551',
 'Q5460288',
 'Q1138408',
 'Q18151267',
 'Q17981446',
 'Q7562391',
 'Q805099',
 'Q4621073',
 'Q1787023',
 'Q6180728',
 'Q471108',
 'Q54319',
 'Q772555',
 'Q639978',
 'Q5764645',
 'Q837573',
 'Q6291502',
 'Q20899554',
 'Q4019621',
 'Q947890',
 'Q25460298',
 'Q160698',
 'Q7777598',
 'Q24391822',
 'Q720504',
 'Q1363733',
 'Q3595742',
 'Q19801909',
 'Q739564',
 'Q524519',
 'Q292350',
 'Q6116949',
 'Q2752724',
 'Q58029',
 'Q6517739',
 'Q818606',
 'Q7574735',
 'Q1113636',
 'Q3898485',
 'Q4191870',
 'Q137443',
 'Q20937',
 'Q11316743',
 'Q7243478',
 'Q314646',
 'Q15139463',
 'Q7430529',
 'Q2109972',
 'Q4761155',
 'Q1269170',
 'Q7232462',
 'Q2387770',
 'Q5818907',
 'Q154701',
 'Q7107088',
 'Q6623135',
 'Q1184780',
 'Q17076813',
 'Q4950648',
 'Q4115894',
 'Q5384577',
 'Q6819242',
 'Q6456627',
 'Q838757',
 'Q2740039',
 'Q6929907',
 'Q1086888',
 'Q188673',
 'Q16859336',
 'Q12800734',
 'Q1157112',
 'Q6554199',
 'Q2438969',
 'Q2057747',
 'Q16824537',
 'Q13572471',

In [17]:
for mode in ["train", "dev", "test"]:
    mode = mode + ".txt"
    path = "./dataset/AliOpenKG500/"+mode
    with open(path) as file:
        samples = file.readlines()

    with open("./dataset/AliOpenKG500/"+mode.replace("txt","tsv"), "w") as file:
        for s in samples:
            t = s.strip().split()
            if len(t) == 1: continue
            h,t,r = t
            file.write('\t'.join([h,r,t]) + '\n')

In [2]:
import os

total_relations = {}
with open("./dataset/AliOpenKG500/relation2id.txt") as file:
    for line in file.readlines():
        line = line.strip().split("\t")
        if len(line) == 1: continue
        r, r_id = line
        total_relations[r_id] = r

with open("./dataset/AliOpenKG500/other_format/relation_map.txt") as file:
    for line in file.readlines():
        line = line.strip().split("\t")
        r_text, r_real_name = line
        # print(r_text)
        for k, v in total_relations.items():
            
            if v == r_text:
                total_relations[k] = r_real_name

with open("./dataset/AliOpenKG500/relation2text.txt", "w") as file:
    for k,v in total_relations.items():
        file.write('\t'.join([k, v]) +'\n')
print(total_relations)



{'0': '规格类型', '1': '薄厚', '2': '是否精酿', '3': '品牌', '4': '适用季节', '5': '产地', '6': '功效', '7': '安全等级', '8': '裤长', '9': '材质', '10': '鞋底材质', '11': '适用年龄', '12': '色泽', '13': '出版社名称', '14': '鞋帮高度', '15': '是否进口', '16': '适用群体', '17': '供暖方式', '18': '适用地区', '19': '成分', '20': '图案', '21': '形状', '22': '特产品类', '23': '分类', '24': '侧翼面料', '25': '样式', '26': '包装方式', '27': '功能', '28': '材质成分', '29': '版型', '30': '设计细节', '31': '裤门襟', '32': '目标人群', '33': '刀头数量', '34': '外底材质', '35': '是否为有机食品', '36': '裤子分类', '37': '组合形式', '38': '安装方式', '39': '口味', '40': '热水器燃气类型', '41': '保质期', '42': '支数', '43': '颜色分类', '44': '产品类别', '45': '表扣款式', '46': '袖长', '47': '填充料', '48': '适用场景', '49': '适用面积', '50': '款式品名', '51': '部位', '52': '领型', '53': '是否带壳', '54': '成分含量', '55': '服装款式细节', '56': '表壳材质', '57': '内文用纸材质', '58': '是否手工', '59': '品牌类型', '60': '毛色', '61': '包装', '62': '帮面材质', '63': '成色', '64': '系列', '65': '贴纸类型', '66': '设计元素', '67': '品牌归属地', '68': '条数', '69': '地市', '70': '添加剂', '71': '流行元素', '72': '质地', '73': '净含量', '74': '衣门襟', '75':

In [11]:
import os

entity2id = {}
relation2id = {}
with open("./dataset/AliOpenKG500/entity2id.txt") as file:
    for line in file.readlines():
        e_name, e_id = line.strip().split("\t")
        entity2id[e_name] = e_id
with open("./dataset/AliOpenKG500/relation2id.txt") as file:
    for line in file.readlines():
        e_name, e_id = line.strip().split("\t")
        relation2id[e_name] = e_id


for mode in ["train", "test", "dev", "test2"]:
    with open(f"./dataset/AliOpenKG500/other_format/{mode}".replace("dev", "valid")) as file:
        data = []
        for line in file.readlines():
            h,r,t = line.strip().split("\t")
            data.append([entity2id[h], relation2id[r], entity2id[t]])

    with open(f"dataset/AliOpenKG500/{mode}.tsv", "w") as file:
        for d in data:
            file.write("\t".join(d) + '\n')
    
    


In [5]:
import os
import random

random.seed(42)

test = []
with open("dataset/AliOpenKG500/dev.tsv") as file:
    test = file.readlines()

test_mini = random.sample(test, 5000)

with open("./dataset/AliOpenKG500/dev_mini.tsv", "w") as file:
    file.writelines(test_mini)
